OFDMA シミュレーション
===

In [116]:
# import
import numpy as np
import sys
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True, floatmode="maxprec_equal")
plt.rcParams['figure.figsize'] = (12,4)

In [117]:
# パラメータ設定
Nuser = 2
Nsc_per_user = 32
Nsc = Nuser * Nsc_per_user
N_GI = 8
NofdmSymbol = 500
Ndata = Nsc_per_user * NofdmSymbol
SNRdB = 20
Lpilot = 2
Delay = np.array([[0]])
Npath = max(Delay.shape)

In [118]:
def rndCode(codeSize, Type):
    """
    codeSize: 生成するランダム系列の大きさ（行数ｘ列数）を指定
    Type: タイプ指定 0: [0,1], 1: [-1, 1]
    """
    
    rndCode = np.random.randn(*codeSize)
    
    if rndCode.ndim > 2:
        sys.exit()
    
    dataSize = rndCode.shape
    if len(dataSize) == 1:
        sys.exit()
    
    if Type == 0:
        rndCode = np.where(rndCode <= 0, 0, 1)
    else:
        rndCode = np.where(rndCode <= 0, -1, 1)
    
    return rndCode

# BPSK変調器
def bpskMod(data):
    """
    data: 列ベクトル
    bpskSymbol： 列ベクトル
    """
    bpskSymbol = data.copy()
    np.place(bpskSymbol, bpskSymbol == 0, -1)
    
    return bpskSymbol

# ガウス雑音生成

def awgn(Pn, rn, cn):
    """
    Pn: 雑音電力
    ｒｎ: 行数
    cn: 列数
    """
    n = np.random.randn(rn, cn) + 1j*np.random.randn(rn, cn)
    n = n * np.sqrt(Pn/2)
    
    return n

# 遅延発生器
def delayGen(delayVec, ssSigMat):
    sigMat = ssSigMat.copy()
    Nsig = delayVec.shape[1]
    
    if Nsig != sigMat.shape[1]:
        sys.exit(1)
    
    for co in range(0, Nsig):
        if delayVec[0,co] > 0:
            roll_num = delayVec[0][co]
            sigMat[:,co] = np.roll(sigMat[:,co],roll_num)
            sigMat[:roll_num, co] = 0
            
    return sigMat

# BPSK復調器
def bpskDem(rSig):
    """
    rSig : 受信信号
    rData:　受信データ
    """
    rData = np.ones(rSig.shape)
    rData[rSig < 0] =  0
    
    return rData

# BER比較器
def ber(data1, data2):
    BER = np.sum(np.abs(data1-data2))/max(data1.shape)
    return BER

In [119]:
# 送信機
data = rndCode([Ndata, Nuser], 0)
bpskSymbol = bpskMod(data)
Nsymbol = max(bpskSymbol.shape)
spOut = np.zeros((Nsc, NofdmSymbol))
for userCo in range(0,Nuser):
    rangeStart = Nsc_per_user * (userCo - 1)
    rangeEnd = Nsc_per_user * userCo
    if rangeStart < 0:
        rangeStart = 0
        rangeEnd = Nsc_per_user
    spOut[rangeStart:rangeEnd, :] = bpskSymbol[:, userCo].reshape(Nsc_per_user, NofdmSymbol, order='F').copy()

In [120]:
# 通信経路
pilotMat = np.ones((Nsc, Lpilot))
ofdmSymbol = np.hstack([pilotMat, spOut])
ofdmSymbol_pilot = np.fft.ifft(ofdmSymbol, axis=0)

gi = ofdmSymbol_pilot[(-1-N_GI):-1,:]
ofdmSymbol_pilot_GI = np.vstack([gi,ofdmSymbol_pilot]);
sOFDM = np.array([ofdmSymbol_pilot_GI.flatten('F')]).T

ofdmSymbolMat = sOFDM * np.ones((1,Npath));
ofdmSymbolMatDelayed = delayGen(Delay, ofdmSymbolMat);
chOut = ofdmSymbolMatDelayed * np.ones((Npath, 1));
Pn = 10 ** (-SNRdB/10) / Nsc;
rSig = chOut + awgn(Pn, max(chOut.shape), 1);

In [121]:
# 受信機
spOutR = rSig.reshape((Nsc+N_GI), (Lpilot+NofdmSymbol), order='F')
spOutR = spOutR[N_GI:, :]
fftOut = np.fft.fft(spOutR, axis=0)
pilotMatRx = fftOut[:, :Lpilot]
ofdmSymbolRx = fftOut[:,Lpilot:]
chCoeff = np.array([np.mean(pilotMatRx, axis=1)]).T
phasesShift = chCoeff/abs(chCoeff) @ np.ones((1, NofdmSymbol))
ofdmSymbolRxCompensated = ofdmSymbolRx * phasesShift.conj()

In [122]:
#BER計算
ofdmSybolRx_user = np.zeros((Nsc, NofdmSymbol))
for userCo in range(0, Nuser):
    rangeStart = Nsc_per_user * (userCo - 1)
    rangeEnd = Nsc_per_user * userCo
    if rangeStart < 0:
        rangeStart = 0
        rangeEnd = Nsc_per_user
    ofdmSymbolRx_user = ofdmSymbolRxCompensated[rangeStart:rangeEnd,:]
    rData = bpskDem(np.array([ofdmSymbolRx_user.flatten('F')]).T)
    BER = ber(np.array([data[:, userCo]]).T, rData)
    print(userCo, BER)

0 0.501375
1 0.0
